In [1]:

import pandas as pd
import numpy as np
import os
path = os.getcwd()

In [2]:

import pandas as pd
import openpyxl
df = pd.read_excel(r"C:\Users\admin\Desktop\MRP.XLSX", engine="openpyxl")
print("Excel資料成功轉換成dataframe...", "\n")

Excel資料成功轉換成dataframe... 



In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', None)
col_list = [i for i in df.columns]
print(col_list, "\n")
print(df.head(10), "\n")



['PLANT', 'MRP AREA', 'PARTNO', 'PART DESCRIPTION', 'VENDOR CODE', 'VENDOR NAME', 'OA Code', 'ABC Ind.', 'MRP Controller', 'REQUEST ITEM', 'PASSDUE', '20230626', '20230703', '20230710', '20230717', '20230724', '20230731', '20230807', '20230814', '20230821', '20230828', '20230904', '20230911', '20230918', 'SEP', 'OCT', 'NOV', 'DEC', 'FUTURE', 'TOTAL', 'PLANT STOCK', 'INSP. STOCK', 'VENDOR PARTNO', 'MRP TYPE', 'UNIT', 'MRP DATE', 'PURCHASING GROUP', 'ROUNDING VALUE', 'LEAD TIME', 'SAFETY STOCK', 'LEAD FREE OF STORAGE LCOATION STOCK', 'Drawing Version', 'Replacement Part Item', 'Replacement Part Description', 'Remark', 'MRP CONTROLLER NAME', 'Range', 'HALOGEN FREE', 'LEAD FREE', 'Purchasing Name', 'MD04 MEMO'] 

  PLANT  MRP AREA      PARTNO               PART DESCRIPTION VENDOR CODE VENDOR NAME     OA Code ABC Ind. MRP Controller REQUEST ITEM  PASSDUE  20230626  20230703  20230710  20230717  20230724  20230731  20230807  20230814  20230821  20230828  20230904  20230911  20230918     SEP 

In [4]:
condition = df['REQUEST ITEM']=='FIRM ORDERS'
dff = df.loc[condition, ['PARTNO', 'REQUEST ITEM','TOTAL']]

dff.columns = ['PARTNO', 'REQUEST ITEM', 'TOTAL_FIRM ORDERS']
dff['REQUEST ITEM'] = 'GROSS REQTS'
dff.index = [i for i in range(len(dff))]
dfc = pd.merge(df, dff, on = ['PARTNO', 'REQUEST ITEM'], how='left')

condition = df['REQUEST ITEM']=='GROSS REQTS'
dfc.loc[condition, 'LEAD_TIME_Week'] = np.ceil(dfc['LEAD TIME']/7)
start_col = dfc.columns.get_loc('PASSDUE')
end_col = dfc.columns.get_loc('FUTURE')
term_week_col = end_col - start_col + 1
print(term_week_col, "\n")

for i in range(len(dfc)):
    if dfc.loc[i, 'LEAD_TIME_Week'] >= term_week_col:
        dfc.loc[i, 'LEAD_TIME_Week_Limit'] = term_week_col
    if dfc.loc[i, 'LEAD_TIME_Week'] < term_week_col:
        dfc.loc[i, 'LEAD_TIME_Week_Limit'] = dfc.loc[i, 'LEAD_TIME_Week']
    

dfc.loc[condition, 'Total_Stock'] = dfc.loc[condition, 'PLANT STOCK'] + dfc.loc[condition, 'INSP. STOCK']

selected_columns = ['PARTNO', 'REQUEST ITEM','Total_Stock'] + list(dfc.loc[:, 'PASSDUE':'FUTURE'].columns)
dfc_result = dfc.loc[condition, selected_columns]
dfc_result.reset_index(drop=True, inplace=True)
gross_cum_list = list(dfc.loc[:, 'PASSDUE':'FUTURE'].columns)
print(dfc_result.head(5), '\n')
print(gross_cum_list, '\n')    


19 

       PARTNO REQUEST ITEM  Total_Stock  PASSDUE  20230626  20230703  20230710  20230717  20230724  20230731  20230807  20230814  20230821  20230828  20230904  20230911  20230918     SEP     OCT      NOV     DEC   FUTURE
0  0341063300  GROSS REQTS       9057.0   2009.0       0.0    1896.0     580.0     236.0     200.0     100.0       0.0       0.0       0.0     401.0      16.0      50.0       0.0   400.0   100.0   2906.0  2548.0  16531.0
1  0341080300  GROSS REQTS       1898.0      0.0       0.0      85.0       0.0       0.0       0.0       0.0     456.0       0.0       0.0     180.0     184.0     148.0       0.0     0.0   732.0    461.0   180.0   2716.0
2  0341092100  GROSS REQTS      10634.0      0.0       0.0    1050.0       0.0    2000.0       0.0       0.0       0.0       0.0       0.0    2000.0       0.0       0.0       0.0  3150.0  2000.0   2000.0  2000.0  13000.0
3  0341097300  GROSS REQTS      10597.0      0.0       0.0      11.0       0.0     382.0       0.0       0.0   

In [5]:
for i in range(len(dfc_result)):
    res = 0
    for j in gross_cum_list:
        res = res + dfc_result.loc[i, j]
        if res > dfc_result.loc[i, 'Total_Stock']:
            dfc_result.loc[i, 'Demand_Week'] = j
            break
        else:
            dfc_result.loc[i, 'Demand_Week'] = 'no demand'

dfc = pd.merge(dfc, dfc_result.reindex(columns=['PARTNO', 'REQUEST ITEM', 'Demand_Week']), on=['PARTNO', 'REQUEST ITEM'], how='left')

print(dfc.head(5), "\n")
# dfc.to_excel(path + "\\" + '456.xlsx', engine="openpyxl", index=False)

  PLANT  MRP AREA      PARTNO               PART DESCRIPTION VENDOR CODE VENDOR NAME     OA Code ABC Ind. MRP Controller REQUEST ITEM  PASSDUE  20230626  20230703  20230710  20230717  20230724  20230731  20230807  20230814  20230821  20230828  20230904  20230911  20230918     SEP     OCT     NOV     DEC   FUTURE    TOTAL  PLANT STOCK  INSP. STOCK      VENDOR PARTNO MRP TYPE UNIT   MRP DATE PURCHASING GROUP  ROUNDING VALUE  LEAD TIME  SAFETY STOCK  LEAD FREE OF STORAGE LCOATION STOCK  Drawing Version Replacement Part Item Replacement Part Description Remark MRP CONTROLLER NAME Range HALOGEN FREE LEAD FREE Purchasing Name MD04 MEMO  TOTAL_FIRM ORDERS  LEAD_TIME_Week  LEAD_TIME_Week_Limit  Total_Stock Demand_Week
0  NTL7       NaN  0341063300  RES SMD 1/10W 15.8Kohm F 0603  0000210032       YAGEO  0000210032        C            010  GROSS REQTS   2009.0       0.0    1896.0     580.0     236.0     200.0     100.0       0.0       0.0       0.0     401.0      16.0      50.0       0.0   400.0

In [6]:
selected_columns = ['PARTNO', 'REQUEST ITEM', 'LEAD_TIME_Week_Limit'] + list(dfc.loc[:, 'PASSDUE':'FUTURE'].columns)

dfcc = dfc.loc[condition, selected_columns]
dfcc.reset_index(drop=True, inplace=True)
start_col = dfc.columns.get_loc('PASSDUE')
for i in range(len(dfcc)):
    res = 0
    for j in range(int(dfcc.loc[i, "LEAD_TIME_Week_Limit"])):
        res = res + dfcc.iloc[i, 3 + j]
    dfcc.loc[i, 'LEAD_TIME_GrossRequest'] = res


dfc = pd.merge(dfc, dfcc[['PARTNO', 'REQUEST ITEM', 'LEAD_TIME_GrossRequest']], on = ['PARTNO', 'REQUEST ITEM'], how='left')

print(dfc.head(5))


  PLANT  MRP AREA      PARTNO               PART DESCRIPTION VENDOR CODE VENDOR NAME     OA Code ABC Ind. MRP Controller REQUEST ITEM  PASSDUE  20230626  20230703  20230710  20230717  20230724  20230731  20230807  20230814  20230821  20230828  20230904  20230911  20230918     SEP     OCT     NOV     DEC   FUTURE    TOTAL  PLANT STOCK  INSP. STOCK      VENDOR PARTNO MRP TYPE UNIT   MRP DATE PURCHASING GROUP  ROUNDING VALUE  LEAD TIME  SAFETY STOCK  LEAD FREE OF STORAGE LCOATION STOCK  Drawing Version Replacement Part Item Replacement Part Description Remark MRP CONTROLLER NAME Range HALOGEN FREE LEAD FREE Purchasing Name MD04 MEMO  TOTAL_FIRM ORDERS  LEAD_TIME_Week  LEAD_TIME_Week_Limit  Total_Stock Demand_Week  LEAD_TIME_GrossRequest
0  NTL7       NaN  0341063300  RES SMD 1/10W 15.8Kohm F 0603  0000210032       YAGEO  0000210032        C            010  GROSS REQTS   2009.0       0.0    1896.0     580.0     236.0     200.0     100.0       0.0       0.0       0.0     401.0      16.0    

In [7]:

dfc.loc[condition, "AMT"] = dfc.loc[condition, "Total_Stock"] + dfc.loc[condition, "TOTAL_FIRM ORDERS"] - dfc.loc[condition, "LEAD_TIME_GrossRequest"]


dfc['PARTNO'] = dfc['PARTNO'].astype(str)
print(dfc.head(5), "\n")

# dfc.to_csv(path + "\\" + '789.csv', index=False, encoding='utf-8-sig')
# dfc.to_csv(path + "\\" + '789.txt', index=False, encoding='utf-8-sig',sep='\t')


  PLANT  MRP AREA      PARTNO               PART DESCRIPTION VENDOR CODE VENDOR NAME     OA Code ABC Ind. MRP Controller REQUEST ITEM  PASSDUE  20230626  20230703  20230710  20230717  20230724  20230731  20230807  20230814  20230821  20230828  20230904  20230911  20230918     SEP     OCT     NOV     DEC   FUTURE    TOTAL  PLANT STOCK  INSP. STOCK      VENDOR PARTNO MRP TYPE UNIT   MRP DATE PURCHASING GROUP  ROUNDING VALUE  LEAD TIME  SAFETY STOCK  LEAD FREE OF STORAGE LCOATION STOCK  Drawing Version Replacement Part Item Replacement Part Description Remark MRP CONTROLLER NAME Range HALOGEN FREE LEAD FREE Purchasing Name MD04 MEMO  TOTAL_FIRM ORDERS  LEAD_TIME_Week  LEAD_TIME_Week_Limit  Total_Stock Demand_Week  LEAD_TIME_GrossRequest     AMT
0  NTL7       NaN  0341063300  RES SMD 1/10W 15.8Kohm F 0603  0000210032       YAGEO  0000210032        C            010  GROSS REQTS   2009.0       0.0    1896.0     580.0     236.0     200.0     100.0       0.0       0.0       0.0     401.0      

In [8]:
select_columns = ['PURCHASING GROUP' , 'REQUEST ITEM'] + list(dfc.loc[:,'PASSDUE':'FUTURE'].columns) + ['TOTAL_FIRM ORDERS',  'Total_Stock']
dfg_1 = dfc.groupby(select_columns[:2])[select_columns[2:]].sum()
dfg_1.reset_index(inplace=True)
print(dfg_1.head(5), '\n')

select_columns = ['PURCHASING GROUP' , 'REQUEST ITEM'] + list(dfc.loc[:,'PASSDUE':'FUTURE'].columns) + ['LEAD_TIME_Week',  'LEAD_TIME_Week_Limit']
dfg_2 = dfc.groupby(select_columns[:2])[select_columns[2:]].min()
dfg_2.reset_index(inplace=True)
print(dfg_2.head(5), '\n')

# 'Demand_Week'+'LEAD_TIME_GrossRequest'+'AMT'

dfg_c = pd.merge(dfg_1, dfg_2[['PURCHASING GROUP' , 'REQUEST ITEM', 'LEAD_TIME_Week',  'LEAD_TIME_Week_Limit']], on=['PURCHASING GROUP' , 'REQUEST ITEM'], how='left')



  PURCHASING GROUP REQUEST ITEM     PASSDUE    20230626    20230703    20230710    20230717    20230724    20230731    20230807    20230814    20230821    20230828    20230904    20230911    20230918         SEP        OCT        NOV        DEC      FUTURE  TOTAL_FIRM ORDERS  Total_Stock
0              BN2  FIRM ORDERS   9625800.0     14000.0    915800.0    173000.0    222000.0   1425600.0    289500.0    233700.0    405700.0    527200.0   1325000.0    267000.0    499000.0    681000.0    536500.0  2495800.0  1694300.0  3195300.0     98000.0                0.0          0.0
1              BN2  GROSS REQTS   3105176.0         0.0   2215936.0    946804.0   2040952.0   2733908.0   1399261.0    871688.0   1356536.0   1942684.0   4422998.0     83168.0         0.0   2029516.0   3314910.0  4072387.0  6843056.0  9530752.0  49190462.0         24624200.0   19609363.0
2              BN2   NET  AVAIL  26103787.0  26117787.0  24817651.0  24043847.0  22224895.0  20916587.0  19806826.0  19168838.0  1821

In [9]:
print(dfg_c.head(5), "\n")
for i in range(len(dfg_c)):
    res = 0
    for j in gross_cum_list:
        res = res + dfg_c.loc[i, j]
        if res > dfg_c.loc[i, 'Total_Stock']:
            dfg_c.loc[i, 'Demand_Week'] = j
            break
        else:
            dfg_c.loc[i, 'Demand_Week'] = 'no demand'
dfg_c.loc[dfg_c['REQUEST ITEM']=='FIRM ORDERS','LEAD_TIME_Week_Limit':'Demand_Week'] = np.nan
dfg_c.loc[dfg_c['REQUEST ITEM']=='NET  AVAIL','LEAD_TIME_Week_Limit':'Demand_Week'] = np.nan
dfg_c.loc[dfg_c['REQUEST ITEM']=='PLAN ORDERS','LEAD_TIME_Week_Limit':'Demand_Week'] = np.nan

# dfg_c.to_csv(path + "\\" + '111.txt', index=False, encoding='utf-8-sig',sep='\t')


  PURCHASING GROUP REQUEST ITEM     PASSDUE    20230626    20230703    20230710    20230717    20230724    20230731    20230807    20230814    20230821    20230828    20230904    20230911    20230918         SEP        OCT        NOV        DEC      FUTURE  TOTAL_FIRM ORDERS  Total_Stock  LEAD_TIME_Week  LEAD_TIME_Week_Limit
0              BN2  FIRM ORDERS   9625800.0     14000.0    915800.0    173000.0    222000.0   1425600.0    289500.0    233700.0    405700.0    527200.0   1325000.0    267000.0    499000.0    681000.0    536500.0  2495800.0  1694300.0  3195300.0     98000.0                0.0          0.0             NaN                   NaN
1              BN2  GROSS REQTS   3105176.0         0.0   2215936.0    946804.0   2040952.0   2733908.0   1399261.0    871688.0   1356536.0   1942684.0   4422998.0     83168.0         0.0   2029516.0   3314910.0  4072387.0  6843056.0  9530752.0  49190462.0         24624200.0   19609363.0             0.0                   0.0
2              BN2 

In [10]:
for i in range(len(dfg_c)):
    res = 0
    try:
        for j in range(int(dfg_c.loc[i, "LEAD_TIME_Week_Limit"])):
            res = res + dfg_c.iloc[i, 2 + j]

        dfg_c.loc[i, 'LEAD_TIME_GrossRequest'] = res
    except:
        continue

condition = dfg_c['REQUEST ITEM'] == 'GROSS REQTS'
dfg_c.loc[condition, "AMT"] = dfg_c.loc[condition, "Total_Stock"] + dfg_c.loc[condition, "TOTAL_FIRM ORDERS"] - dfg_c.loc[condition, "LEAD_TIME_GrossRequest"]

order_set = {"GROSS REQTS":0, "FIRM ORDERS":1, "PLAN ORDERS":2, "NET  AVAIL":3}
dfg_c['order_set'] = dfg_c['REQUEST ITEM'].map(order_set)
dfg_c.sort_values(by=['order_set'], ascending=True, inplace=True)

print(dfg_c.head(10), '\0')

   PURCHASING GROUP REQUEST ITEM      PASSDUE    20230626     20230703     20230710     20230717     20230724     20230731     20230807     20230814     20230821     20230828    20230904     20230911     20230918          SEP          OCT          NOV          DEC        FUTURE  TOTAL_FIRM ORDERS   Total_Stock  LEAD_TIME_Week  LEAD_TIME_Week_Limit Demand_Week  LEAD_TIME_GrossRequest           AMT  order_set
57              BNG  GROSS REQTS  34872768.00  3669390.00  21177923.00  26665787.00  15371912.00  20950663.00  26112523.00  17242060.00  13701921.00  10969602.00  15331488.00  21700078.0  12079445.00  13132526.00  21734758.00  75612297.00  84775753.00  65237591.00  2.467114e+08       2.049500e+08  1.519731e+08             4.0                   4.0    20230807             86385868.00  2.705373e+08          0
21              BN7  GROSS REQTS   2561924.60        0.00   1097878.00    548468.00   1821001.60   2020716.39    799662.00    485816.00    823488.00   2070316.00   2631898.61    

In [11]:


dfc = pd.concat([dfc, dfg_c], axis=0)

dfc.to_csv(path + "\\" + '111.txt', index=False, encoding='utf-8-sig',sep='\t')

